In [14]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("Csv-Üzeri-SQL") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [15]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("D:\\Datasets\\OnlineRetail.csv")

In [16]:
df.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [17]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



Örneğin yukarıda UnitPrice'ın  Float olmasına ihtiyaç var

Spark'ın çıkarımı bize her zaman yetmez ellerimizle kendi şemamızı yapalım
Önce gerekli kütüphaneleri içeri alalım

In [18]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, DateType,DoubleType

# Elle şema oluşturma

In [22]:
manual_schema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", StringType(), True),
    StructField("UnitPrice", FloatType(), True),
    StructField("CustomerID", IntegerType(), True),
    StructField("Country", StringType(), True)

])

# Veriye elle hazırlanan şema ile tekrar okuma

In [23]:
df2 = spark.read \
.option("header","True") \
.option("sep",";") \
.schema(manual_schema) \
.csv("D:\\Datasets\\OnlineRetail.csv")

In [24]:
df2.show(5)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|     null|     null|       null|    null|       null|     null|      null|   null|
|     null|     null|       null|    null|       null|     null|      null|   null|
|     null|     null|       null|    null|       null|     null|      null|   null|
|     null|     null|       null|    null|       null|     null|      null|   null|
|     null|     null|       null|    null|       null|     null|      null|   null|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
only showing top 5 rows



UnitPrice'da sıkıntı var. "." yerine "," olduğu için schma string'den float'a dönüştüremiyor. Çözüm bulmak gerekli.

# ÇÖZÜM
Çözüm olarak veriyi okurken "," ile "." yı değiştirerek okuyalım ve dataframe'i tekrar , ve . değişmiş
şekilde diske yazalım. Diskten okurken düzeltilmiş csv den okuyalım ve elle hazırlanmış şemamızı kullanalım.

## Veriyi temizleyerek okuma

In [26]:
from pyspark.sql import functions as F

df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("D:\\Datasets\\OnlineRetail.csv") \
.withColumn("UnitPrice",F.regexp_replace(F.col("UnitPrice"), ",","."))

Temiz dataframe'i diske yazalım

In [27]:
df \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("sep",";") \
    .option("header","true") \
    .csv("D:\\Datasets\\OnlineRetail2")

Temizlenmiş olarak diske yazdığımız dosyayı tekrar okuyalım. Bu sefer manual_schema verelim.

In [30]:
df_temiz = spark.read \
.option("header","True") \
.schema(manual_schema) \
.option("sep",";") \
.csv("D:\\Datasets\\OnlineRetail2") \

In [31]:
df_temiz.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows

